In [1]:
# 0-500
import requests
from bs4 import BeautifulSoup
import sqlite3
import numpy as np
import pandas as pd
import time
import random
# Google增強DataFrame的渲染和導出
# from google.colab import data_table
# data_table.enable_dataframe_formatter()


count=367

def 連接資料庫(db):

    global conn,cursor,dfDividend,ids

    # 連接到資料庫A ====================================================================
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    sql = '''select * from dividend t'''
    dfDividend = pd.read_sql(sql,conn)


    # 連接到資料庫B
    sqlStock = '''select * from stock t'''
    dfStock = pd.read_sql(sqlStock,conn)
    ids = dfStock['code'][count:500]
    # ==================================================================================

def 年報表爬蟲(stocks,counta):
    global df2

    delay_choices = [40, 50, 60, 30, 70]  #延遲的秒數   
    delay = random.choice(delay_choices)  #隨機選取秒數

    for id in stocks:
        print(id)  

        # 爬個股資料
        url = f'https://goodinfo.tw/tw/StockDividendPolicy.asp?STOCK_ID={id}'
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.33'}
        res = requests.get(url, headers = headers,timeout=20)
        res.encoding = "utf-8"
        soup = BeautifulSoup(res.text,"lxml") 
        data = soup.select_one("#divDetail")
        dfs = pd.read_html(data.prettify())
        df = dfs[0]
        df.columns = df.columns.get_level_values(3)

        # 整理表格
        df2 = df.copy()
        filter1 = "股 利 政 策"
        filter2 = '股利  發放  年度'
        filter3 = '∟'
        filter4 = '累計'

        # 刪除【股利政策】=============================================
        if filter1 in df2["股利  發放  年度"].values:
            dfdrop1 = df2[df2["股利  發放  年度"]==filter1].index
            df2 = df2.drop(dfdrop1)
        else:
            pass

        # 刪除【股利發放年度】=========================================
        if filter2 in df2["股利  發放  年度"].values:
            dfdrop2 = df2[df2["股利  發放  年度"]==filter2].index
            df2 = df2.drop(dfdrop2)
        else:
            pass

        # 刪除【特殊字符】=============================================
        if filter3 in df2["股利  發放  年度"].values:
            dfdrop3 = df2[df2["股利  發放  年度"]==filter3].index
            df2 = df2.drop(dfdrop3)
        else:
            pass

        # 刪除【累計】=============================================
        if filter4 in df2["股利  發放  年度"].values:
            dfdrop4 = df2[df2["股利  發放  年度"]==filter4].index
            df2 = df2.drop(dfdrop4)
        else:
            pass

        # 選中需要的部分
        df2 = df2.loc[:,['股利  發放  年度','股利  合計']]
        df2.insert(0,'代碼',id)
        df2.columns = ['代碼','股利發放年度','股利合計']

        # 寫入資料庫
        for index,row in df2.iterrows():
            try:
                cursor.execute(
                """INSERT OR IGNORE INTO dividend 
                (代碼,股利發放年度,股利合計)
                values(?,?,?)""",
                (row['代碼'],
                row['股利發放年度'],
                row['股利合計'])
                )
                conn.commit()
            except:
                pass

        counta += 1
        print(f'已完成{counta}次')
        time.sleep(delay)
      

if __name__ == "__main__":
    連接資料庫('goodinfoRevenue(年)1.db')
    年報表爬蟲(ids,count)

    print('爬蟲完畢')
    conn.close()

6257


AttributeError: 'NoneType' object has no attribute 'prettify'